# References

https://www.youtube.com/watch?v=8Jktm-Imt-I&list=PLGB9meziqbzr3Ipb2ZzIsDGiIeqD_Kzc4&index=3&t=0s
    
https://examples.pyviz.org/glaciers/glaciers.html 
    
    
## distraction
    - https://github.com/karolzak/ipyplot
    
    - https://github.com/plotly/dash-datashader/blob/master/app.py

# rEQUISITES

conda install -c pyviz pandas=0.24 param=1.9.0 panel=0.5.1 holoviews=1.12.2 geoviews=1.6.2 datashader=0.7.0

In [ ]:
import numpy as np, pandas as pd, cartopy.crs as ccrs
import holoviews as hv, geoviews as gv, datashader as ds

from colorcet import bmy
from holoviews.operation.datashader import rasterize, datashade

hv.extension('bokeh', width=100)

In [ ]:
! conda --help

In [ ]:
#! pip3 install --user cartopy

In [ ]:
df = pd.read_csv('data/oggm_glacier_explorer.csv')
df['latdeg'] = df.cenlat

df.tail()

In [ ]:
#! wget https://raw.githubusercontent.com/pyviz-demos/glaciers/master/data/oggm_glacier_explorer.csv

In [ ]:
#! mv ogg* data; head data/oggm_glacier_explorer.csv

In [ ]:
data = gv.Points(df, [('cenlon', 'Longitude'), ('cenlat', 'Latitude')],
                     [('avg_prcp', 'Annual Precipitation (mm/yr)'),
                      ('area_km2', 'Area'), ('latdeg', 'Latitude (deg)'),
                      ('avg_temp_at_mean_elev', 'Annual Temperature at avg. altitude'), 
                      ('mean_elev', 'Elevation')])
total_area = df.area_km2.sum()
print(data, len(data), total_area)

In [ ]:
data = gv.Dataset(gv.operation.project_points(data))

In [ ]:
geo_kw    = dict(aggregator=ds.sum('area_km2'), x_sampling=1000, y_sampling=1000)
elev_kw   = dict(cmap='#7d3c98')
temp_kw   = dict(num_bins=50, adjoin=False, normed=False, bin_range=data.range('avg_temp_at_mean_elev'))
prcp_kw   = dict(num_bins=50, adjoin=False, normed=False, bin_range=data.range('avg_prcp'))

size_opts = dict(min_height=400, min_width=600, responsive=True)
geo_opts  = dict(size_opts, cmap=bmy, global_extent=False, logz=True, colorbar=True)
elev_opts = dict(size_opts, show_grid=True)
temp_opts = dict(size_opts, fill_color='#f1948a', default_tools=[], toolbar=None, alpha=1.0)
prcp_opts = dict(size_opts, fill_color='#85c1e9', default_tools=[], toolbar=None, alpha=1.0)

In [ ]:
geo_bg = gv.tile_sources.EsriImagery.options(alpha=0.6, bgcolor="black")
geopoints = data.to(gv.Points, ['cenlon', 'cenlat'], ['area_km2'], []).options(**geo_opts).redim.range(area_km2=(0, 3000))

(geo_bg*rasterize(geopoints, **geo_kw).options(**geo_opts) + 
 datashade(data.to(hv.Scatter, 'mean_elev','latdeg', []), **elev_kw).options(**elev_opts) + 
 data.hist('avg_temp_at_mean_elev', **temp_kw).options(**temp_opts) +
 data.hist('avg_prcp',              **prcp_kw).options(**prcp_opts)).cols(2)